# Experimento 00

Primer experimento creado por Federico Bayle

## 1. Generar una variable de NDVI por fecha

In [50]:
import pandas as pd


In [51]:
def indice_normalizado(banda_1, banda_2):
    num = banda_1-banda_2
    den = banda_1+banda_2
    return(num/den)

In [52]:
df = pd.read_csv('dataset_train.csv')

df['NDVI_2020-10-01'] = indice_normalizado(df['B8_2020-10-01'], df['B4_2020-10-01'])
df['NDVI_2020-11-01'] = indice_normalizado(df['B8_2020-11-01'], df['B4_2020-11-01'])
df['NDVI_2020-12-01'] = indice_normalizado(df['B8_2020-12-01'], df['B4_2020-12-01'])
df['NDVI_2021-01-01'] = indice_normalizado(df['B8_2021-01-01'], df['B4_2021-01-01'])
df['NDVI_2021-02-20'] = indice_normalizado(df['B8_2021-02-20'], df['B4_2021-02-20'])
df['NDVI_2021-03-17'] = indice_normalizado(df['B8_2021-03-17'], df['B4_2021-03-17'])

In [53]:
df[df.cultivo==1].filter(regex='NDVI_').describe()

,NDVI_2020-10-01,NDVI_2020-11-01,NDVI_2020-12-01,NDVI_2021-01-01,NDVI_2021-02-20,NDVI_2021-03-17
count,2549.000000,2549.000000,2549.000000,2549.000000,2549.000000,2549.000000
mean,0.558033,0.475763,0.182881,0.494853,0.873275,0.774080
std,0.231945,0.201371,0.054860,0.252372,0.080971,0.148459
min,0.138041,0.136402,0.109205,0.117954,0.480078,0.304921
25%,0.424544,0.346322,0.149232,0.293658,0.844697,0.701466
50%,0.627376,0.502745,0.165710,0.433376,0.909296,0.834598
75%,0.729854,0.625668,0.195931,0.729409,0.929292,0.883873
max,0.910012,0.861702,0.443776,0.945135,0.950285,0.930769


## 2. Filtrar el dataset para quedarnos con las bandas NDVI para cada fecha

In [54]:
df_filter = df.filter(regex=r'(NDVI|id|cultivo)')
print(df_filter.columns)
df_filter.head()

Index(['id', 'cultivo', 'NDVI_2020-10-01', 'NDVI_2020-11-01',
       'NDVI_2020-12-01', 'NDVI_2021-01-01', 'NDVI_2021-02-20',
       'NDVI_2021-03-17'],
      dtype='object')


,id,cultivo,NDVI_2020-10-01,NDVI_2020-11-01,NDVI_2020-12-01,NDVI_2021-01-01,NDVI_2021-02-20,NDVI_2021-03-17
0,0,1,0.163823,0.160763,0.180151,0.831622,0.930923,0.596334
1,1,1,0.161071,0.160036,0.181945,0.826462,0.930088,0.596491
2,2,1,0.158433,0.164615,0.180225,0.796710,0.932804,0.600372
3,3,1,0.167779,0.156943,0.181055,0.826950,0.930699,0.614431
4,4,1,0.159408,0.161499,0.187386,0.799875,0.928182,0.605181


## 3. Separar el dataset de entrenamiento en 3 partes

In [55]:
from sklearn.model_selection import train_test_split

In [56]:
# Let's say we want to split the data in 80:10:10 for train:valid:test dataset
train_size=0.8

X = df_filter.drop(['id', 'cultivo'], axis=1).copy()
y = df_filter['cultivo']

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(7124, 6)
(7124,)
(890, 6)
(890,)
(891, 6)
(891,)


(None, None)

## 4. Usando los datasets de los puntos 2.1 y 2.2, ajustar una búsqueda por grid search para un Random Forest utilizando cross validation con 5 folds

In [57]:
grid = { 
    'n_estimators': [2,4,8],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [2,4,6],
    'criterion' :['gini', 'entropy'],
    'random_state' : [18]
}

In [58]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf_cv = GridSearchCV(estimator=RandomForestClassifier(), param_grid=grid, cv= 5, verbose=2, refit=True)
rf_cv.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] criterion=gini, max_depth=2, max_features=sqrt, n_estimators=2, random_state=18 
[CV]  criterion=gini, max_depth=2, max_features=sqrt, n_estimators=2, random_state=18, total=   0.0s
[CV] criterion=gini, max_depth=2, max_features=sqrt, n_estimators=2, random_state=18 
[CV]  criterion=gini, max_depth=2, max_features=sqrt, n_estimators=2, random_state=18, total=   0.0s
[CV] criterion=gini, max_depth=2, max_features=sqrt, n_estimators=2, random_state=18 
[CV]  criterion=gini, max_depth=2, max_features=sqrt, n_estimators=2, random_state=18, total=   0.0s
[CV] criterion=gini, max_depth=2, max_features=sqrt, n_estimators=2, random_state=18 
[CV]  criterion=gini, max_depth=2, max_features=sqrt, n_estimators=2, random_state=18, total=   0.0s
[CV] criterion=gini, max_depth=2, max_features=sqrt, n_estimators=2, random_state=18 
[CV]  criterion=gini, max_depth=2, max_features=sqrt, n_estimators=2, random_state=18, total=   0.0s

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s



[CV] criterion=gini, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18 
[CV]  criterion=gini, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=gini, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18 
[CV]  criterion=gini, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18, total=   0.0s
[CV] criterion=gini, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18 
[CV]  criterion=gini, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=gini, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18 
[CV]  criterion=gini, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=gini, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18 
[CV]  criterion=gini, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=gini, max_depth=2, max_features=sqrt, n_estimator

[CV]  criterion=gini, max_depth=4, max_features=log2, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=gini, max_depth=4, max_features=log2, n_estimators=4, random_state=18 
[CV]  criterion=gini, max_depth=4, max_features=log2, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=gini, max_depth=4, max_features=log2, n_estimators=4, random_state=18 
[CV]  criterion=gini, max_depth=4, max_features=log2, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=gini, max_depth=4, max_features=log2, n_estimators=4, random_state=18 
[CV]  criterion=gini, max_depth=4, max_features=log2, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=gini, max_depth=4, max_features=log2, n_estimators=8, random_state=18 
[CV]  criterion=gini, max_depth=4, max_features=log2, n_estimators=8, random_state=18, total=   0.1s
[CV] criterion=gini, max_depth=4, max_features=log2, n_estimators=8, random_state=18 
[CV]  criterion=gini, max_depth=4, max_features=log2, n_estimator

[CV]  criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18 
[CV]  criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18 
[CV]  criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=8, random_state=18 
[CV]  criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=8, random_state=18, total=   0.1s
[CV] criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=8, random_state=18 
[CV]  criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=8, random_state=18, total=   0.2s
[CV] criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=8, random_state=18 
[CV]  criterion=entropy, max_depth=

[CV]  criterion=entropy, max_depth=4, max_features=log2, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=entropy, max_depth=4, max_features=log2, n_estimators=4, random_state=18 
[CV]  criterion=entropy, max_depth=4, max_features=log2, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=entropy, max_depth=4, max_features=log2, n_estimators=4, random_state=18 
[CV]  criterion=entropy, max_depth=4, max_features=log2, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=entropy, max_depth=4, max_features=log2, n_estimators=4, random_state=18 
[CV]  criterion=entropy, max_depth=4, max_features=log2, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=entropy, max_depth=4, max_features=log2, n_estimators=4, random_state=18 
[CV]  criterion=entropy, max_depth=4, max_features=log2, n_estimators=4, random_state=18, total=   0.1s
[CV] criterion=entropy, max_depth=4, max_features=log2, n_estimators=8, random_state=18 
[CV]  criterion=entropy, max_depth=

[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:   17.9s finished


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 4, 6],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [2, 4, 8], 'random_state': [18]},
             verbose=2)

In [59]:
rf_cv.best_params_

{'criterion': 'gini',
 'max_depth': 6,
 'max_features': 'sqrt',
 'n_estimators': 8,
 'random_state': 18}

In [60]:
from sklearn.metrics import confusion_matrix
y_test_pred = rf_cv.predict(X_test)
confusion_matrix(y_test, y_test_pred)

array([[243,  21,   0,   5,   0,   0,   0],
       [ 28, 176,   2,   0,   0,   0,   0],
       [  0,   8, 254,   2,   3,   0,   0],
       [  4,   4,   3,  47,   1,   0,   0],
       [  0,   1,   9,   1,  40,   0,   0],
       [  0,   0,   0,   0,   0,   2,   0],
       [  0,   0,   5,   0,   0,   0,  32]], dtype=int64)

## 6. Reproducir la creación del dataset para dataset_predict.csv

In [61]:
df_test = pd.read_csv('dataset_predict.csv')
df_test['NDVI_2020-10-01'] = indice_normalizado(df_test['B8_2020-10-01'], df_test['B4_2020-10-01'])
df_test['NDVI_2020-11-01'] = indice_normalizado(df_test['B8_2020-11-01'], df_test['B4_2020-11-01'])
df_test['NDVI_2020-12-01'] = indice_normalizado(df_test['B8_2020-12-01'], df_test['B4_2020-12-01'])
df_test['NDVI_2021-01-01'] = indice_normalizado(df_test['B8_2021-01-01'], df_test['B4_2021-01-01'])
df_test['NDVI_2021-02-20'] = indice_normalizado(df_test['B8_2021-02-20'], df_test['B4_2021-02-20'])
df_test['NDVI_2021-03-17'] = indice_normalizado(df_test['B8_2021-03-17'], df_test['B4_2021-03-17'])

In [62]:
df_test = df_test.filter(regex=r'(NDVI|id)')
print(df_test.columns)
df_test.head()

Index(['id', 'NDVI_2020-10-01', 'NDVI_2020-11-01', 'NDVI_2020-12-01',
       'NDVI_2021-01-01', 'NDVI_2021-02-20', 'NDVI_2021-03-17'],
      dtype='object')


,id,NDVI_2020-10-01,NDVI_2020-11-01,NDVI_2020-12-01,NDVI_2021-01-01,NDVI_2021-02-20,NDVI_2021-03-17
0,381,0.605525,0.336082,0.166603,0.680446,0.900632,0.864944
1,382,0.591148,0.347675,0.159486,0.671239,0.907263,0.872921
2,383,0.609422,0.359781,0.161820,0.615292,0.909219,0.888999
3,384,0.624434,0.350834,0.168781,0.590148,0.914425,0.888913
4,385,0.600228,0.325905,0.169361,0.616181,0.914606,0.891449


In [63]:
X_pred = df_test.drop(['id'], axis=1)
df_test['cultivo'] = rf_cv.predict(X_pred)
df_test.head()

,id,NDVI_2020-10-01,NDVI_2020-11-01,NDVI_2020-12-01,NDVI_2021-01-01,NDVI_2021-02-20,NDVI_2021-03-17,cultivo
0,381,0.605525,0.336082,0.166603,0.680446,0.900632,0.864944,2
1,382,0.591148,0.347675,0.159486,0.671239,0.907263,0.872921,2
2,383,0.609422,0.359781,0.161820,0.615292,0.909219,0.888999,1
3,384,0.624434,0.350834,0.168781,0.590148,0.914425,0.888913,1
4,385,0.600228,0.325905,0.169361,0.616181,0.914606,0.891449,1


## 7. Generar el archivo clases.csv con el formato solicitado

In [64]:
df_test[['id', 'cultivo']].to_csv('00_clases.csv', index=False)